In [122]:
import pandas as pd
import unicodedata
from rutas import CSV_BIBLIOTECAS, CSV_CINES, CSV_MUSEOS


### 2 Procesamiento de Datos
<span>Normalizar toda la información de Museos, Salas de Cine y Bibliotecas Populares, para crear una única tabla que contenga:</span>

<style>
    ul{columns:4;}
    div{margin-top:25px;}
</style>
<div class='lista'>
    <ul>
        <li>cod_localidad</li>
        <li>id_provincia</li>
        <li>id_departamento</li>
        <li>categoría</li>
        <li>provincia</li>
        <li>localidad</li>
        <li>nombre</li>
        <li>domicilio</li>
        <li>código postal</li>
        <li>número de teléfono</li>
        <li>mail</li>
        <li>web</li>
    </ul>
</div>


#### 2.1 Normalizando y consolidando campos

In [164]:
# Leer csvs

raw_df_bibliotecas = pd.read_csv(CSV_BIBLIOTECAS)
raw_df_museos = pd.read_csv(CSV_MUSEOS)
raw_df_cines = pd.read_csv(CSV_CINES)

In [185]:
def normalizar_campos(csv_file):
    # Leer csv
    raw_df = pd.read_csv(csv_file)
    # Normalizar campos
    raw_df.columns = raw_df.columns.str.lower()
    raw_df.columns = raw_df.columns.str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')
    return raw_df

raw_df_bibliotecas = normalizar_campos(CSV_BIBLIOTECAS)
raw_df_museos = normalizar_campos(CSV_MUSEOS)
raw_df_cines = normalizar_campos(CSV_CINES)

In [186]:

def normalizar_valores(df:pd.DataFrame):
    new_df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    new_df.fillna('s/d', inplace=True)
    def no_tildes(texto):
        new_value = unicodedata.normalize('NFKD',texto).encode('ascii',errors='ignore').decode('utf-8')
        return new_value
    new_df = new_df.applymap(lambda x: no_tildes(x) if isinstance(x, str) else x)
    return new_df
    

In [187]:
raw_df_bibliotecas = normalizar_valores(raw_df_bibliotecas)
raw_df_museos = normalizar_valores(raw_df_museos)
raw_df_cines = normalizar_valores(raw_df_cines)

In [188]:
raw_df_museos['provincia'].unique()

array(['Buenos Aires', 'Catamarca', 'Chaco', 'Chubut',
       'Ciudad Autonoma de Buenos Aires', 'Cordoba', 'Corrientes',
       'Entre Rios', 'Formosa', 'Jujuy', 'La Pampa', 'La Rioja',
       'Mendoza', 'Misiones', 'Neuquen', 'Rio Negro', 'Salta', 'San Juan',
       'San Luis', 'Santa Cruz', 'Santa Fe', 'Santiago del Estero',
       'Tierra del Fuego, Antartida e Islas del Atlantico Sur', 'Tucuman'],
      dtype=object)

In [169]:
sorted(raw_df_bibliotecas['provincia'].unique())

['Buenos Aires',
 'Catamarca',
 'Chaco',
 'Chubut',
 'Ciudad Autonoma de Buenos Aires',
 'Cordoba',
 'Corrientes',
 'Entre Rios',
 'Formosa',
 'Jujuy',
 'La Pampa',
 'La Rioja',
 'Mendoza',
 'Misiones',
 'Neuquen',
 'Rio Negro',
 'Salta',
 'San Juan',
 'San Luis',
 'Santa Cruz',
 'Santa Fe',
 'Santiago del Estero',
 'Tierra del Fuego',
 'Tucuman']

In [42]:
def nuevos_campos(df:pd.DataFrame):
    match df.categoria[0]:
        case 'Salas de cine':
            new_df = df[['cod_localidad', 'id_provincia', 'id_departamento', 'categoria', 'provincia', 'localidad', 'nombre', 'direccion', 'cp']].copy()
            new_df[['número de teléfono', 'mail']] = 's/d'
            new_df['web'] = df['web']
        
        case 'Espacios de Exhibición Patrimonial':
            new_df = df[['cod_loc', 'idprovincia', 'iddepartamento', 'categoria', 'provincia', 'localidad', 'nombre', 'direccion', 'cp', 'telefono', 'mail', 'web']]
        
        case 'Bibliotecas Populares':
            new_df = df[['cod_loc', 'idprovincia', 'iddepartamento', 'categoria', 'provincia', 'localidad', 'nombre', 'domicilio', 'cp', 'telefono', 'mail', 'web']]
    
    return new_df

# Seleccionar columnas requeridas en cada dataframe
df_bibliotecas = nuevos_campos(raw_df_bibliotecas)
df_museos = nuevos_campos(raw_df_museos)
df_cines = nuevos_campos(raw_df_cines)


In [7]:
# Renombrando columnas
def rename_cols(df):
    '''
    Crea un diccionario a partir de un zip entre las columnas actuales 
    del dataframe y una lista con sus correspondientes nuevos nombres
    Pasar como argumento el dataframe con las columnas a renombrar
    '''
    # Crear lista de nomenclaturas para nueva tabla
    columnas = ['cod_localidad', 'id_provincia', 'id_departamento', 'categoría', 
                'provincia', 'localidad', 'nombre', 'domicilio', 'código postal', 
                'número de teléfono', 'mail', 'web']
    diccionario = dict(zip(df.columns, columnas))
    df = df.rename(columns=diccionario)
    return df


In [8]:
# Crear dataframes con columnas renombradas
df_bibliotecas = rename_cols(df_bibliotecas)
df_museos = rename_cols(df_museos)
df_cines = rename_cols(df_cines)

#### 2.2 Normalizando datos

In [9]:
df_bibliotecas.fillna('s/d', inplace=True)
df_museos.fillna('s/d', inplace=True)
df_cines.fillna('s/d', inplace=True)

#### 2.3 Crear tabla única

In [10]:
consolidado = pd.concat([df_bibliotecas,df_museos,df_cines], ignore_index=True)

### 3 Procesando los datos conjuntos


#### 3.1 Generar una tabla con la siguiente información:
- Cantidad de registros totales por categoría
- Cantidad de registros totales por fuente
- Cantidad de registros por provincia y categoría

In [11]:
registros_categoria = pd.DataFrame(consolidado.groupby(by= 'categoría')['categoría'].count())
registros_categoria.columns=['total_registros']

In [12]:
registros_categoria.reset_index(inplace=True)

In [13]:
registros_categoria

,categoría,total_registros
0,Bibliotecas Populares,2017
1,Espacios de Exhibición Patrimonial,1182
2,Salas de cine,316


In [189]:
campos_agregaciones = ['provincia','categoria','fuente']
bibliotecas_aggr = raw_df_bibliotecas[campos_agregaciones]
cines_aggr = raw_df_cines[campos_agregaciones]
museos_aggr = raw_df_museos[campos_agregaciones]
lista_dfs_aggr = [bibliotecas_aggr,museos_aggr,museos_aggr]
unificado_aggr = pd.concat(lista_dfs_aggr, ignore_index=True)

In [190]:
def limpiar_tdf(provincia:pd.Series):
    return provincia.split(',')[0]
unificado_aggr['provincia'] = unificado_aggr['provincia'].apply(limpiar_tdf)

In [194]:
unificado_aggr.columns

Index(['provincia', 'categoria', 'fuente'], dtype='object')

In [193]:
unificado_aggr.groupby(by='fuente')['fuente'].count()

fuente
Area Gestion Cultural de la Municipalidad de Colonia Caroya               8
CNMLH - Enlace SInCA                                                      2
CNMLH - Ente Cultural de Tucuman                                          2
CONABIP                                                                2010
DNPyM                                                                  2096
Direccion Provincial de Patrimonio Cultural - Enlace SInCA                2
Direccion de Cultura de Villa Allende                                     4
Direccion de Cultura y de Educacion de la Municipalidad de Unquillo       2
Gob. Pcia.                                                                1
Gobierno de la Provincia                                                 28
Gobierno de la Provincia de Chubut / SInCA 2013                           6
Gobierno de la Provincia de Jujuy                                         8
Gobierno de la Provincia de La Pampa                                      2
Gobie

In [14]:
registros_fuente = pd.concat([raw_df_bibliotecas,raw_df_museos,raw_df_cines], ignore_index=True)
registros_fuente = pd.DataFrame(registros_fuente.groupby(by='fuente')['fuente'].count())
registros_fuente.columns=['total_registros']

In [43]:
pd.concat([raw_df_bibliotecas,raw_df_museos,raw_df_cines], ignore_index=True).columns

Index(['cod_loc', 'idprovincia', 'iddepartamento', 'observacion', 'categoria',
       'subcategoria', 'provincia', 'departamento', 'localidad', 'nombre',
       'domicilio', 'piso', 'cp', 'cod_tel', 'telefono', 'mail', 'web',
       'informacion adicional', 'latitud', 'longitud', 'tipolatitudlongitud',
       'fuente', 'tipo_gestion', 'ano_inicio', 'ano_actualizacion',
       'observaciones', 'direccion', 'cod_area', 'info_adicional',
       'jurisdiccion', 'ano_inauguracion', 'actualizacion', 'cod_localidad',
       'id_provincia', 'id_departamento', 'tipo_latitud_longitud', 'sector',
       'pantallas', 'butacas', 'tipo_de_gestion', 'espacio_incaa'],
      dtype='object')

In [18]:
registros_fuente.reset_index(inplace=True)
registros_fuente

,fuente,total_registros
0,CNMLH - Enlace SInCA,1
1,CNMLH - Ente Cultural de Tucumán,1
2,CONABIP,2010
3,DNPyM,1048
4,Dirección Provincial de Patrimonio Cultural - ...,1
5,Dirección de Cultura de Villa Allende,2
6,Dirección de Cultura y de Educación de la Muni...,1
7,Gob. Pcia.,1
8,Gobierno de la Provincia,14
9,Gobierno de la Provincia de Chubut / SInCA 2013,3


In [28]:
registros_fuente.fuente.unique()

array(['CNMLH - Enlace SInCA', 'CNMLH - Ente Cultural de Tucumán',
       'CONABIP', 'DNPyM',
       'Dirección Provincial de Patrimonio Cultural - Enlace SInCA',
       'Dirección de Cultura de Villa Allende',
       'Dirección de Cultura y de Educación de la Municipalidad de Unquillo',
       'Gob. Pcia.', 'Gobierno de la Provincia',
       'Gobierno de la Provincia de Chubut / SInCA 2013',
       'Gobierno de la Provincia de Jujuy',
       'Gobierno de la Provincia de La Pampa',
       'Gobierno de la Provincia de La Rioja',
       'Gobierno de la Provincia de Mendoza',
       'Gobierno de la Provincia de Neuquén',
       'Gobierno de la Provincia de Salta',
       'Gobierno de la Provincia de San Juan', 'Gobierno de la provincia',
       'INCAA / SInCA', 'Municipalidad de Puerto Madryn',
       'Municipalidad de Río Gallegos - Red de Ciudades Creativas',
       'Municipalidad de Santa Fe - Red de Ciudades Creativas', 'RCC',
       'RCC- Córdoba', 'Red de Espacios Culturales Vicente

In [25]:
registros_provincia_categoria = pd.DataFrame(consolidado.groupby(by=['provincia', 'categoría'])['provincia'].count())
registros_provincia_categoria.columns = ['total_registros']

In [26]:
registros_provincia_categoria.reset_index(inplace=True)
registros_provincia_categoria

,provincia,categoría,total_registros
0,Buenos Aires,Bibliotecas Populares,543
1,Buenos Aires,Espacios de Exhibición Patrimonial,152
2,Buenos Aires,Salas de cine,101
3,Catamarca,Bibliotecas Populares,38
4,Catamarca,Espacios de Exhibición Patrimonial,32
...,...,...,...
68,Tierra del Fuego,Salas de cine,3
69,"Tierra del Fuego, Antártida e Islas del Atlánt...",Espacios de Exhibición Patrimonial,14
70,Tucumán,Bibliotecas Populares,37
71,Tucumán,Espacios de Exhibición Patrimonial,26


In [27]:
registros_provincia_categoria.provincia.unique()

array(['Buenos Aires', 'Catamarca', 'Chaco', 'Chubut',
       'Ciudad Autónoma de Buenos Aires', 'Corrientes', 'Córdoba',
       'Entre Ríos', 'Formosa', 'Jujuy', 'La Pampa', 'La Rioja',
       'Mendoza', 'Misiones', 'Neuquén', 'Neuquén\xa0', 'Río Negro',
       'Salta', 'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe',
       'Santa Fé', 'Santiago del Estero', 'Tierra del Fuego',
       'Tierra del Fuego, Antártida e Islas del Atlántico Sur', 'Tucumán'],
      dtype=object)

#### 3.2 Procesar la información de cines
Crear una tabla que contenga:
- Provincia
- Cantidad de pantallas
- Cantidad de butacas
- Cantidad de espacios INCAA

In [137]:
tabla_cines = raw_df_cines[['provincia', 'pantallas', 'butacas', 'espacio_incaa']].copy()
tabla_cines['espacio_incaa'] = (tabla_cines['espacio_incaa'] == 'Si').astype(int)
tabla_cines = tabla_cines.groupby(by='provincia').sum()